In [1]:
import time
import pandas as pd
from ast import literal_eval

import warnings
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

import sys
sys.path.insert(1, 'C:/Users/tom/projects/skill-skeleton/utils/')
sys.path.insert(2, 'C:/Users/tom/projects/skill-skeleton/utils/neo4j/')
from connection import Neo4jConnection
import query as query
import manage as manage
import kb_util


def add_skills(rows, batch_size=10000):
    # Adds skill nodes to the Neo4j graph as a batch job.  

    query = '''UNWIND $rows AS row   
    MERGE (s:Skill {name: row.skills})
    RETURN count(*) as total
    '''     
    
    return insert_data(query, rows, batch_size)


def fix_new_skills(rows, batch_size=10000):
    
    parameters = {'cat': "BD-ML-AI-Courses"}   
        
    query = '''UNWIND $rows AS row    
    MATCH (s:Skill {name: row.skills})
    WHERE s.category IS NULL
    SET s.category = $cat   
    RETURN count(*) as total
    '''      
    
    return insert_data(query, rows, batch_size, parameters)


def add_courses(rows, batch_size=10000):
    # Adds courses nodes to the Neo4j db as a batch job.    

    parameters = {'annotated_by': "manual"}   

    query = '''UNWIND $rows AS row
    MERGE (c:Course {title: row.title, url: row.url, date: row.date, course_info: row.course_info, start_time: row.start_time, language: row.language, location_detail: row.location_detail, target_group: row.target_group, subscription_limit: row.subscription_limit, constraints: row.constraints, price: row.price, details: row.details, sub_title: row.sub_title, intro: row.intro, full_body: row.full_body, data: row.data, annotated_by: $annotated_by})
    RETURN count(*) as total
    '''
    return insert_data(query, rows, batch_size, parameters)


def add_links(rows, batch_size=5000):
    # Adds profile nodes and (:Course)--(:Skill)
    # relationships to the Neo4j graph as a batch job.  (Note the smaller batch
    # size due to the fact that this function is adding much more data than the
    # add_skills() function.)

    query = '''
    UNWIND $rows as row   
    WITH row
    MATCH (s:Skill {name: row.skills})   
    MATCH (c:Course {url: row.url})
    MERGE (c)-[:OFFERS]->(s)
    RETURN count(distinct s) as total
    '''

    return insert_data(query, rows, batch_size)


def insert_data(query, rows, batch_size = 10000, parameters=None):
    # Function to handle the updating the Neo4j database in batch mode.

    total = 0
    batch = 0
    start = time.time()
    result = None

    while batch * batch_size < len(rows):

        if parameters is None:
            param = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')}        
        else:
            param = parameters | {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')}

        print(param)
        
        res = conn.query(query, parameters=param)
        total += res[0]['total']
        batch += 1
        result = {"total":total, "batches":batch, "time":time.time()-start}
        print(result)

    return result


def get_skills(df):    
           
    df['skills'] = df['skills'].apply(kb_util.fix)
    
    return df[['url','skills']]


def get_courses(df):    
    
    df = df.fillna("NA")       
    return df.drop_duplicates()


def populate_db(df):
    
    df = df[df['skills']!="[]"] 
    df = df[df["language"]=="English"]    
      
     
    courses = get_courses(df)
    
    skills = get_skills(df)    
    skills['skills'] = skills['skills'].apply(literal_eval)    
    exploded_skills = skills.explode('skills').drop_duplicates().dropna()
            
    add_courses(courses)    
    add_skills(exploded_skills)
    add_links(exploded_skills)
    fix_new_skills(exploded_skills)
    
    
def configure_db():
    conn.query('CREATE CONSTRAINT courses IF NOT EXISTS FOR (c:Course) REQUIRE c.url IS UNIQUE')
    conn.query('CREATE TEXT INDEX course_text_index_url IF NOT EXISTS FOR (c:Course) ON (c.url)')
    conn.query('CREATE TEXT INDEX skill_text_index_name IF NOT EXISTS FOR (s:Skill) ON (s.name)')


def setup_db(file):
    df = pd.read_csv(file,delimiter='|',dtype={'url':str, 'title':str, 'skills':str, 'language':str})
    
    # Specify the columns to consider when looking for duplicates
    columns = ['title','url','date']

    # Find and drop duplicates, this is normally already done after downloading the data
    df = df.drop_duplicates(subset=columns)    
    
    configure_db()
    populate_db(df)


def delete_course_skill_link():
    conn.query('MATCH (c:Course)-[e:OFFERS]->(s:Skill) delete e')


def delete_courses():
    conn.query('MATCH (c:Course) DETACH DELETE c')
    
    
def delete_skills(category):    
    parameters = {'cat': category}    
    conn.query('MATCH (s:Skill) WHERE s.category = $cat DETACH DELETE s', parameters=parameters)


def get_all_courses():
    query_string = '''
    MATCH (c:Course) RETURN c.title as title, c.url as url, c.date as date, c.course_info as course_info, c.start_time as start_time, c.language as language, c.location_detail as location_detail, c.target_group as target_group, c.subscription_limit as subscription_limit, c.constraints as constraints, c.price as price, c.details as details, c.sub_title as sub_title, c.intro as intro, c.full_body as full_body
    '''
    
    return pd.DataFrame([dict(_) for _ in conn.query(query_string)])    


def get_all_courses_for_NER():
    query_string = '''
    MATCH (c:Course) RETURN c.url as url, c.title as title, c.course_info as course_info, c.full_body as full_body, c.details as details, c.sub_title as sub_title, c.intro as intro
    '''
    
    return pd.DataFrame([dict(_) for _ in conn.query(query_string)])    


def refresh_matcher_and_scores(conn):
    manage.refresh_all_profile_skill_scores(conn)
    
    skill_without_duplicates = query.get_all_skills(conn)
    kb_util.create_matcher_from_db('C:/Users/tom/projects/skill-skeleton/models/NER/finalized_matcher.sav',skill_without_duplicates, save=True)
  
   
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="neo4j",              
                       pwd="neo4jneo4j")   

In [2]:
#delete_course_skill_link()
#delete_courses()
#delete_skills("BD-ML-AI-Courses")
setup_db("vaia-final-data-IT2.csv")
refresh_matcher_and_scores(conn)

{'annotated_by': 'manual', 'rows': [{'title': 'Gender and AI, a feminist perspective', 'url': 'https://www.vaia.be/en/courses/gender-and-ai-a-feminist-perspective', 'date': '22/03/2024', 'course_info': 'lecture-Brussels-FARI', 'start_time': '25 Mar 2024 17:30 - 19:00', 'language': 'English', 'location_detail': 'FARI  Cantersteen 16, 1000 Brussels', 'target_group': 'Target audience: Administrators, industry professionals and AI enthusiasts are invited to join this session.', 'subscription_limit': 'NA', 'constraints': 'NA', 'price': 'Gratis', 'details': 'https://www.vub.be/en/event/ai-happy-hour-gender-and-ai-feminist-perspective', 'sub_title': '25 Mar 2024 17:30 - 19:00', 'intro': 'More info  Share this course  ', 'full_body': ' For this session  Anastasia Karagianni  Doctoral Candidate at the LSTS Department of the Law and Criminology Faculty of VUB  will examine gender issues in AI from a feminist perspective. Through constant interaction with the audience  key definitions of AI will 